# import modules

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import os
import csv
import pandas as pd

In [2]:
def static():
    global meal_translator
    global web_url
    global driver
    meal_translator = {'B':'Breakfast','L':"Lunch",'D':'Dinner','O':"Others"}
    web_url = 'https://www.calorieking.com/us/en/foods/'
    def start_program():
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        driver = webdriver.Chrome(options=options)
        return driver
    driver = start_program()
    

In [3]:
def create_file():
    global columns
    columns = ['Date', 'Meal_Time' , 'Brand', 'Food', 'Calories', 'Fat', 'Carbs', 'Fiber', 'Protein']
    if os.path.isfile('food_recoder.csv') == False:
        with open('food_recoder.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(columns)
            

In [4]:
def get_key_word():
    driver.get(web_url)
    time.sleep(0.2)
    search_terms = input('Please enter a brand.\t')
    print(f'search for the terms: \t{search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    brands = driver.find_elements_by_class_name('jss5')
    global result_lst
    result_lst = [ i.find_element_by_class_name('MuiTypography-root').text for i in brands]
    if 'iOS'in str(result_lst):
        print('Please try another search tearms for brand.')
        get_key_word(driver, web_url)
    return result_lst

In [5]:
def get_brand():
    def check_brand():
        for i in result_lst:
            yield(i)
    brands = check_brand()
    correct = 'F'
    global target_brand
    while correct == 'F':
        try:
            brand_candidate = next(brands)
            correct = input(f'If {brand_candidate} the brand you are looking for? T/F:\t').upper()
            if correct =='T':
                target_brand = str(brand_candidate)
                return target_brand 
                break
        except: 
            print('Please try another search tearms for brand.')
            get_key_word()
            target_brand = get_brand()
            return target_brand 
            break   

In [6]:
def get_key_word_checked():
    driver.get(web_url)
    time.sleep(0.2)
    global meal_terms
    meal_terms = input('Please enter the main meal ingrediant.\t').upper()
    search_terms = target_brand + ' ' + meal_terms
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    foods = driver.find_elements_by_class_name('jss374')
    global food_lst
    food_lst = [ i.text for i in foods]
    if 'iOS'in str(food_lst):
        print('Please try another search tearms for brand.')
        get_key_word_checked()
    return food_lst

In [7]:
# narrow down to the specific dish and get the dish's info link
def get_food():
    count=0
    def check_food(food_lst):
        for i in food_lst:
            yield(i)
    foods = check_food(food_lst)
    correct = 'F'
    global target_food_url
    global target_food
    while correct == 'F':
        try:
            food_candidate = next(foods)
            correct = input(f'If {food_candidate} the meal you are looking for? T/F:\t').upper()
            if correct =='T':
                target_food = str(food_candidate)
                url = driver.find_elements_by_class_name('MuiListItem-button')[count].get_attribute('href')
                target_food_url = (url)
                return target_food_url, target_food
                break
            else:
                count=count+1
        except: 
            print('Please try another search tearms for meal.')
            get_key_word_checked()
            target_food_url, target_food = get_food()
            return target_food_url, target_food
            break

In [8]:
def get_food_info():
    driver.get(target_food_url)
    infos = driver.find_element_by_class_name('jss374')
    infos = infos.text.split('\n')
    
    calories = infos[0].split(' ')[0]
    fat = infos[6].split(' ')[0]
    carbs = infos[8].split(' ')[0]
    fiber = infos[10].split(' ')[0]
    protein = infos[12].split(' ')[0]
    
    return calories, fat, carbs, fiber, protein

In [9]:
def get_time():
    global date
    date = input('Please enter the date. mm/dd/yyyy.\t')
    print('Please enter the meal time.')
    global meal_time_transfered
    meal_time = input('B---Breakfast\nL---Lunch\nD---Dinner\nO---Others').upper()
    if meal_time in ['B','L','D','O']:
        meal_time_transfered = meal_translator[meal_time]
    else:
        print('Please enter the meal time again.')
        get_time()
    return date, meal_time_transfered

In [10]:
def get_inputs(): 
    static()
    result_lst = get_key_word()
    target_brand = get_brand()
    food_lst = get_key_word_checked()
    target_food_url, target_food = get_food()
    calories, fat, carbs, fiber, protein = get_food_info()
    date, meal_time_transfered = get_time()
    return date, meal_time_transfered, target_brand, target_food, calories, fat, carbs, fiber, protein

In [11]:
def run_again():
    run = input('Have another meal to record? T/F \t').upper()
    if run == 'T':
        main()
    else:
        print('Thanks for using this software.')

In [12]:
# check for results
def check_result():
    record = pd.read_csv('food_recoder.csv')
    print(f'Total amount of {len(record)} data in the file.')
    print(record.head())
    

In [13]:
class food:

    def __init__(self, date, meal_time_transfered, target_brand, target_food,
                 calories, fat, carbs, fiber, protein):
        self.date = date
        self.meal_time_transfered = meal_time_transfered
        self.target_brand = target_brand 
        self.target_food = target_food
        self.calories = calories
        self.fat = fat
        self.carbs = carbs
        self.fiber = fiber
        self. protein = protein
        
    def show_info(self): 
        data = [self.date, self.meal_time_transfered, self.target_brand, 
                                  self.target_food, self.calories, self.fat, self.carbs, self.fiber, self.protein]
        columns = ['Date', 'Meal_Time' , 'Brand', 'Food', 'Calories', 'Fat', 'Carbs', 'Fiber', 'Protein']
        meal_info = pd.DataFrame([data], columns = columns)
        return meal_info

        
    def save_result(self):
        with open('food_recoder.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([self.date, self.meal_time_transfered, self.target_brand, 
                            self.target_food, self.calories, self.fat, self.carbs, self.fiber, self.protein])


In [14]:
def main():
    create_file()
    date, meal_time_transfered, target_brand, target_food, calories, fat, carbs, fiber, protein = get_inputs()
    meal = food(date, meal_time_transfered, target_brand, target_food, calories, fat, carbs, fiber, protein)
    meal.show_info()
    meal.save_result()
    run_again()
    check_result()

In [15]:
main()

Please enter a brand.	kfc
search for the terms: 	kfc
If KFC the brand you are looking for? T/F:	t
Please enter the main meal ingrediant.	chicken
search for the terms: KFC CHICKEN
If Chicken Hot Wings the meal you are looking for? T/F:	f
If Extra Crispy Chicken Tenders the meal you are looking for? T/F:	f
If Original Recipe Chicken Drumstick, bone in the meal you are looking for? T/F:	f
If Original Recipe Chicken Breast, bone in the meal you are looking for? T/F:	f
If Original Recipe Chicken Thigh, bone in the meal you are looking for? T/F:	t
Please enter the date. mm/dd/yyyy.	01/02/2021
Please enter the meal time.
B---Breakfast
L---Lunch
D---Dinner
O---Otherso
Have another meal to record? T/F 	f
Thanks for using this software.
Total amount of 3 data in the file.
         Time  Meal_Time       Brand                                    Food  \
0  01/02/2021  Breakfast    Popeye's       Classic or Spicy Chicken Sandwich   
1  01/02/2021     Dinner  Applebee's     Sirloin Steak (8 oz), with